In [1]:
import sys; sys.path.append('../../')
import numpy as np
from DataPreparation.DataPreparation import read_data
from utils import load_hyperparameters, save_model, get_metrics
from sklearn import svm
from sklearn.metrics import classification_report
from mlpath import mlquest as mlq
from ModelAnalysis import cross_validation
import pickle
from sklearn.ensemble import BaggingClassifier
mlq.start_quest("SVMBagging", table_dest="../../", log_defs=True)
model_name = 'SVM'

# <font color="aqua">SVM</font> Model

### Read the data

In [2]:
# x_data_d, y_data_d = mlq.l(read_data)(encode='Frequency', selected=False)
x_data_d, y_data_d = mlq.l(read_data)(kind='Numerical', selected=False)

# x_data_d =x_data_d [['Phys_Act','Height','Weight']]
opt_params = load_hyperparameters(model_name)

### Initiate model

In [3]:
# clf = mlq.l(svm.SVC)(**opt_params)
clf = mlq.l(BaggingClassifier)(base_estimator=svm.SVC(**opt_params), n_estimators=3, max_samples=0.9, max_features=0.5)

In [4]:
print(opt_params)

{'C': 1186.865625052541, 'gamma': 0.551065306122449, 'kernel': 'linear'}


### Evaluate Model Bias

In [5]:
clf.fit(x_data_d, y_data_d)
y_pred = clf.predict(x_data_d)
train_metrics = classification_report(y_data_d, y_pred, digits=3)
train_acc, train_wf1 = get_metrics(train_metrics)
print(train_metrics)

              precision    recall  f1-score   support

           0      0.721     0.816     0.765       152
           1      0.524     0.406     0.458       160
           2      0.646     0.772     0.703       324
           3      0.920     0.840     0.878       544

    accuracy                          0.759      1180
   macro avg      0.703     0.708     0.701      1180
weighted avg      0.765     0.759     0.759      1180



### Evaluate Model Generalization

In [6]:
kfold=cross_validation(clf, x_data_d, y_data_d, k=[5], n_repeats=[1], random_state=1, loo=False)

val_wf1 = kfold.get(f'10-Repeated 10-fold')[0] 
# repeated_10fold_report= kfold.get(f'10-Repeated 10-fold')[1]
# print(f'10-Repeated 10-fold Report:\n {repeated_10fold_report}')

# loo_wf1=loo.get('loo_wf1')
# loo_report=loo.get('loo_report')
# print(f'Leave-One-Out Report:\n {loo_report}')

KeyboardInterrupt: 

#### Save Model

In [ ]:
save_model(model_name, clf)

### Tracking

In [ ]:
mlq.log_metrics(train_acc, train_wf1, val_wf1, sv)
mlq.end_quest()
mlq.show_logs("SVM", table_dest="../../", last_k=10)